In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

/home/pandu/anaconda3/envs/llm-copy/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import pickle
import numpy as np
import faiss
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from transformers import AutoTokenizer, AutoModel
import torch

class PDFEmbeddingStore:
    def __init__(self, model_name='intfloat/multilingual-e5-large'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def text_to_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        return embeddings

    def normalize_embeddings(self, embeddings):
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        normalized_embeddings = embeddings / norms
        return normalized_embeddings

    def parse_pdf_and_create_embeddings(self, pdf_file_path):
        pdf_reader = PdfReader(pdf_file_path)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() or ""  # Fallback to empty string if None

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        chunks = text_splitter.split_text(text=text)
        embeddings = np.vstack([self.text_to_embedding(chunk) for chunk in chunks if chunk.strip()])
        return embeddings, chunks

    def create_faiss_index(self, embeddings):
        embeddings = self.normalize_embeddings(embeddings)  # Normalize before creating the index
        d = embeddings.shape[1]  # Dimension of embeddings
        index = faiss.IndexFlatIP(d)  # Use IndexFlatIP for cosine similarity
        index.add(embeddings)
        return index

In [3]:
# Functions for loading the FAISS index and chunks, and for searching the index
def load_faiss_index(index_path):
    return faiss.read_index(index_path)

def load_chunks(chunks_path):
    with open(chunks_path, "rb") as f:
        chunks = pickle.load(f)
    return chunks

def search_index(query, pdf_embedding_store, index, top_k=5):
    query_embedding = pdf_embedding_store.text_to_embedding(query)
    query_embedding = pdf_embedding_store.normalize_embeddings(query_embedding)
    distances, indices = index.search(query_embedding.astype(np.float32), top_k)
    return distances, indices

def get_text_results(indices, chunks):
    return [chunks[idx] if idx >= 0 else "Not found" for idx in indices[0]]

In [4]:
pdf_embedding_store = PDFEmbeddingStore()

# Process PDF and create embeddings
pdf_file_path = 'UUD45 ASLI.pdf'  # Specify your PDF file path here
embeddings, chunks = pdf_embedding_store.parse_pdf_and_create_embeddings(pdf_file_path)

# Creating and storing the FAISS index
index = pdf_embedding_store.create_faiss_index(embeddings)
store_name = os.path.basename(pdf_file_path).rsplit('.', 1)[0]  # Remove file extension
faiss.write_index(index, f"{store_name}.index")

# Additionally, save the chunks for later retrieval
chunks_path = f"{store_name}_chunks.pkl"
with open(chunks_path, "wb") as f:
    pickle.dump(chunks, f)

# Load the stored FAISS index and chunks for inference
index = load_faiss_index(f"{store_name}.index")
chunks = load_chunks(chunks_path)

# User's query for inference
user_query = "bendera negara Indonesia ialah?"  # Update with the actual query

# Search for the most similar chunks/documents
distances, indices = search_index(user_query, pdf_embedding_store, index, top_k=5)

# Retrieve and print the natural language text results
text_results = get_text_results(indices, chunks)
print("Text results for the top K similar chunks/documents:")
for i, text in enumerate(text_results):
    print(f"{i+1}: \n================================================================================\n{text}")

Text results for the top K similar chunks/documents:
1: 
orang  
banyak dikuasai oleh negara.  
(3) Bumi dan air dan kekayaan alam yang terk andung didalamnya dikuasai oleh negara 
dan 
dipergunakan untuk sebesar -besar kemakmuran rakyat.  
 
Pasal 34  
Fakir miskin dan anak -anakyang terlantar dipelihara oleh negara.  
 
 
 
 
 
BAB XV  
BENDERA DAN BAHASA  
Pasal 35  
Bendera Negara Indonesia ialah Sang Merah Puti h. 
Pasal 36  
Bahasa negara ialah Bahasa Indonesia.   
BAB XVI  
PERUBAHAN UNDANG -UNDANG DASAR  
Pasal 37  
(1) Untuk mengubah Undang -Undang Dasar sekurang -kurangnya 2/3 dari pada jumlah 
anggota Majelis Permusyawaratan Rakyat harus hadir.  
(2) Putusan diambil dengan persetujuan sekurang -kurangnya 2/3 dari pada jumlah 
anggota  
Majelis Permusyawaratan Rakyat yang hadir.  
 
ATURAN PERALIHAN  
Pasal 1  
Panitia Persiapan Kemerdekaan Indonesia mengatur dan menyelenggarakan kepindahan  
pemerintahan kepada Pemerintah Indonesia .  
 
Pasal II  
Segala badan negara dan pe

In [21]:
def prepare_llm_messages(user_query, chunks, indices):
    """
    Prepares messages for LLM chat including the context from retrieved chunks.
    """
    # Retrieve and prepare the context from chunks
    context_texts = get_text_results(indices, chunks)
    context_message = "\n\n".join(context_texts)  # Joining multiple chunks with new lines
    print(f"Context of the query is: \n{context_message}")
    system_prompt = f"""The you are helpful assistant that only answer in Bahasa. Your answer is based on the context given. The context is: {context_message}
    Answer the question directly without giving any reasoning or notes.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    
    return messages

In [22]:
from llama_cpp import Llama

# Initialize LLM
llm = Llama(
    model_path=r"/home/pandu/Documents/LLM/llama.cpp/models/mistral-7b-instruct-v0.2.Q5_K_M.gguf", 
    verbose=True,
    chat_format="llama-2",
    n_ctx=2048)  # Adjust chat_format as necessary

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /home/pandu/Documents/LLM/llama.cpp/models/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llam

In [28]:
# User's query
user_query = "Apa bentuk pemerintahan Indonesia?"

# Perform semantic search to get indices of relevant chunks
distances, indices = search_index(user_query, pdf_embedding_store, index, top_k=3)

# Prepare messages for LLM with context from retrieved chunks
messages = prepare_llm_messages(user_query, chunks, indices)

# Generate response using LLM
response = llm.create_chat_completion(messages=messages)
print("==================================================")
print(response['choices'][0]['message']['content'])

Context of the query is: 
dengan lain perkataan dinamis. Berhubung dengan itu, hanya aturan -aturan pokok saja 
harus  
ditetapka n dalam Undang -Undang Dasar, sedangkan hal -hal yang perlu untuk  
menyelenggarakan aturan -aturan pokok itu harus diserahkan kepada undang -undang.  
 
SISTEM PEMERINTAHAN NEGARA  
 
Sistem pemerintahan negara yang ditegaskan dalam Undang -Undang Dasar ialah:  
 
I.Indo nesia ialah negara yang berdasar atas hukum (rechtsstaat).   
1. Negara Indonesia berdasar atas hukum, (rechtsstaat), tidak berdasarkan  
kekuasaan belaka (Machtsstaat).  
 
II Sistem Konstitusional.  
 
2. Pemerintahan berdasar atas sistem konstitusi (hukum dasar) tidak bersifat  
absolutisme (kekuasaan yang tidak terbatas).  
 
III. Kekuasaan Negara yang tertinggi di tangan Majelis Permusyawaratan Rakyat  
(Die gezatnte Staatgewalt liegi allein bei der Majelis).  
 
3. Kedaulatan Rakyat dipegang oleh suatu badan, bernama Maj elis Permusyawaratan 
Rakyat,

besar dari ada haluan negara.  
 

Llama.generate: prefix-match hit


 Indonesia adalah negara yang berbasis atas hukum (rechtsstaat) dengan sistem konstitusional, dimana kekuasaan negara tertinggi berada di tangan Majelis Permusyawaratan Rakyat. Presiden Republik Indonesia memegang kekuasaan pemerintahan dan dipompa oleh Wakil Presiden. Presiden membentuk undang-undang dengan persetujuan Dewan Perwakilan Rakyat dan menetapkan peraturan pemerintah untuk menjalankannya. Presiden harus seorang Indonesia asli, dipilih oleh Majelis Permusyawaratan Rakyat dengan suara terbanyak, dan memegang jabatannya selama masa lima tahun. Jika Presiden mati, berhenti atau tidak dapat melakukan kewajibannya dalam masa jabatannya, maka Wakil Presiden akan menjadi pengganti sampai habis waktunya. Indonesia tidak mempunyai daerah yang berstatus negeri dalam lingkungannya, namun dibagi dalam daerah propinsi dan daerah yang lebih kecil. Di daerah-daerah otonom atau yang bersifat daerah administrasi belaka akan diadakan badan perwakilan daerah, dimana pemerintahan disusun atas d


llama_print_timings:        load time =   24705.82 ms
llama_print_timings:      sample time =     233.57 ms /   487 runs   (    0.48 ms per token,  2085.04 tokens per second)
llama_print_timings: prompt eval time =   45849.54 ms /   842 tokens (   54.45 ms per token,    18.36 tokens per second)
llama_print_timings:        eval time =   81383.64 ms /   486 runs   (  167.46 ms per token,     5.97 tokens per second)
llama_print_timings:       total time =  128589.19 ms
